In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [2]:
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')

from nltk import word_tokenize #tokenizer
nltk.download('punkt') #sentence tokenizer
nltk.download('averaged_perceptron_tagger')

from bs4 import BeautifulSoup
import html as ihtml

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Load the data

In [3]:
train_csv = pd.read_csv('train.csv', error_bad_lines=False).fillna(' ')
test_csv = pd.read_csv('test.csv', error_bad_lines=False).fillna(' ')
display(train_csv.head())

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
# aggregation of toxic labels
train = train_csv
train['general_toxic'] = ((train['toxic'] + train['severe_toxic'] + train['obscene'] + train['threat'] + train['insult'] + train['identity_hate']) > 0).astype(int)
train = train.drop(columns=['toxic', 'severe_toxic', 'severe_toxic', 'threat', 'insult', 'identity_hate', 'obscene'])
display(train[200:250].head()) 

,id,comment_text,general_toxic
200,007f127033d66db5,"""\n\n Link to Sacramento \n\nI placed a wikipe...",0
201,007f1839ada915e6,Your blatant POV pushing \n\nNeither of you gu...,1
202,00803f08f55bdcad,"""\nReliable sources indicate otherwise. Please...",0
203,008198c5a9d85a8e,"I did research thank you very much, and it is ...",0
204,00822d0d01752c7e,"""\n\nA """"TotallyDisputed"""" notice would probab...",0


# Aggregate toxic labels

In [5]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,4), dpi=120)
train.groupby('general_toxic')['general_toxic'].count().plot(kind='barh', ax=ax)


<AxesSubplot:ylabel='general_toxic'>

# Defining functions

In [6]:
def lemmatize(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text])
def remove_punc(text):
    return ''.join([word for word in text if word not in string.punctuation])

In [7]:
def remove_html(text):
    text = BeautifulSoup(ihtml.unescape(text), "lxml").text
    text = re.sub(r"\s+", " ", text)
    return text
remove_html(train.loc[0, "comment_text"])

"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

# Pre-processing

In [8]:
# Remove HTML
train['clean'] = train['comment_text'].apply(lambda x: remove_html(x))
train['clean'].head()

c:\users\patrick\appdata\local\programs\python\python37\lib\site-packages\bs4\__init__.py:421: MarkupResemblesLocatorWarning: "http://en.wikipedia.org/wiki/Wikipedia_talk:No_original_research/archive15#YouTube_art_as_primary_source" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\patrick\appdata\local\programs\python\python37\lib\site-packages\bs4\__init__.py:421: MarkupResemblesLocatorWarning: "http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\patrick\appdata\local\programs\python\python37\lib\site-packages\bs4\__init__.py:421: MarkupResemblesLoca

0    Explanation Why the edits made under my userna...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    " More I can't make any real suggestions on im...
4    You, sir, are my hero. Any chance you remember...
Name: clean, dtype: object

In [10]:
# Remove Punctuations
train['clean'] = train['clean'].apply(lambda x: remove_punc(x))
#train['clean'].head()
display(train.head()) 

,id,comment_text,general_toxic,clean
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,Daww He matches this background colour Im seem...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,Hey man Im really not trying to edit war Its j...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,More I cant make any real suggestions on impr...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [13]:
 #lemmatizer = WordNetLemmatizer() 
 #train['lemmatized'] = train['clean'].apply(lambda x: lemmatize(x))
 #display(train.head())

In [11]:
train_text = word_tokenize(train['clean'][0])
nltk.pos_tag(train_text)

[('Explanation', 'NN'),
 ('Why', 'WRB'),
 ('the', 'DT'),
 ('edits', 'NNS'),
 ('made', 'VBN'),
 ('under', 'IN'),
 ('my', 'PRP$'),
 ('username', 'JJ'),
 ('Hardcore', 'NNP'),
 ('Metallica', 'NNP'),
 ('Fan', 'NNP'),
 ('were', 'VBD'),
 ('reverted', 'VBN'),
 ('They', 'PRP'),
 ('werent', 'VBP'),
 ('vandalisms', 'NNS'),
 ('just', 'RB'),
 ('closure', 'NN'),
 ('on', 'IN'),
 ('some', 'DT'),
 ('GAs', 'NNP'),
 ('after', 'IN'),
 ('I', 'PRP'),
 ('voted', 'VBD'),
 ('at', 'IN'),
 ('New', 'NNP'),
 ('York', 'NNP'),
 ('Dolls', 'NNP'),
 ('FAC', 'NNP'),
 ('And', 'CC'),
 ('please', 'VB'),
 ('dont', 'JJ'),
 ('remove', 'VB'),
 ('the', 'DT'),
 ('template', 'NN'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('talk', 'NN'),
 ('page', 'NN'),
 ('since', 'IN'),
 ('Im', 'NNP'),
 ('retired', 'VBD'),
 ('now892053827', 'NNS')]

In [10]:
#Using CountVectorizer
# pipeline = Pipeline([('vectorizer', CountVectorizer(max_df=0.5,
#                                                    min_df=4,
#                                                    tokenizer='[a-zA-Z]+')),
#                     ('clf', MultinomialNB())], verbose=True)
# pipeline.fit(train['comment_text'], train['general_toxic'])

In [11]:
#Using TfidVectorizer; COPY PASTED FROM https://www.kaggle.com/piumallick/toxic-comments-sentiment-analysis
#for reference only
vectorizer = TfidfVectorizer(ngram_range=(1, 3),
                             max_df=0.5,
                             min_df=4,
                             max_features=1000)
vector_space_model = vectorizer.fit_transform(train['comment_text'].values.astype('U').tolist()) # converting the dtype object to unicode string 
n_comments = vector_space_model.shape[0]
print('%d Total Comments' % n_comments)

NameError: name 'train' is not defined